Category = Mens Clothing

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import random

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)

try:
    # Navigate to the main page
    driver.get("https://www.amazon.com")

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "nav-hamburger-menu"))).click()   # Main Dropbar
    time.sleep(2) 

    see_all_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//li/a/div[contains(text(), 'See all')]"))   # See all button
    )
    see_all_button.click()
    time.sleep(2) 

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a/div[contains(text(), \"Men's Fashion\")]"))).click() # Mens fashion button
    time.sleep(2)  

    # Click on "Clothing" using the new provided path
    clothing_link = "/s?bbn=16225019011&rh=i%3Aspecialty-aps%2Cn%3A7141123011%2Cn%3A16225019011%2Cn%3A1040658&ref_=nav_em__nav_desktop_sa_intl_clothing_0_2_13_2"
    driver.get(f"https://www.amazon.com{clothing_link}")
    time.sleep(5)  # Adjust based on page load time

    data = []
    product_urls = []  # New list to store product URLs
    gender = ""  # Initialize gender before the loop
    main_category = ""
    subcategory = ""  # Initialize subcategory before the loop

    while len(data) < 500:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        products = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for product in products:
            if len(data) >= 500:
                break

            title = product.find('span', class_='a-size-base-plus')
            
            if title:
                product_link = product.find('a', class_='a-link-normal')['href']
                full_link = f"https://www.amazon.com{product_link}"
                product_urls.append(full_link)  # Add product URL to the list
                
                data.append({
                    'product_url': full_link,  # Add the product URL
                    'title': title.text.strip(),
                    'main_category': main_category,
                    'subcategory': subcategory,
                    'gender': gender,
                })
                
                # Click on the product link
                driver.get(full_link)
                time.sleep(3)  # Adjust based on page load time
                
                # Scrape product details using provided XPath
                try:
                    product_details_element = driver.find_element(By.XPATH, "//*[contains(@id, 'detailBullets')]")
                    product_details = product_details_element.text.strip()
                    data[-1]['product_details'] = product_details
                    print(f"Scraped Product Details - Product: {title.text.strip()}, Details: {product_details}")

                    # Scrape product description using provided XPath
                    product_description_element = driver.find_element(By.XPATH, "(//*[contains(@id, 'bylineInfo')])[1]")
                    product_description = product_description_element.text.strip()
                    data[-1]['product_description_brand_site'] = product_description
                    print(f"Scraped Product Description (Brand Site) - Product: {title.text.strip()}, Description: {product_description}")

                    # Scrape product image URLs using provided XPath
                    product_image_elements = driver.find_elements(By.XPATH, "//*[contains(@id, 'a-autoid')]/div/div[1]/img")
                    product_image_urls = [element.get_attribute('src') for element in product_image_elements]
                    data[-1]['product_image_urls'] = product_image_urls
                    print(f"Scraped Product Image URLs - Product: {title.text.strip()}, Image URLs: {product_image_urls}")

                    # Scrape price using provided XPath
                    price_element = driver.find_element(By.XPATH, "//*[@id='corePrice_feature_div']//span")
                    price = price_element.text.strip()
                    data[-1]['price'] = price
                    print(f"Scraped Price - Product: {title.text.strip()}, Price: {price}")

                except Exception as e:
                    print(f"Error scraping product details, brand site description, image URLs, or price: {e}")

                # Scrape review details
                try:
                    reviews = driver.find_elements(By.XPATH, "//div[contains(@data-hook, 'review')]")
                    for review in reviews[:1]:  # Get only the first review
                        reviewer_name_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[1]/a/div[2]/span")
                        reviewer_name = reviewer_name_element.text

                        review_title_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[2]/a/span[2]")
                        review_title = review_title_element.text

                        star_ratings_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[2]/a/i")

                        star_rating_text = star_ratings_element.get_attribute("class")
                        star_rating = star_rating_text if star_rating_text else None

                        data[-1].setdefault('reviews', []).append({
                            'reviewer_name': reviewer_name,
                            'star_rating': star_rating,
                            'review_title': review_title
                        })

                        # Print review details
                        print(f"Scraped Review - Product: {title.text.strip()}, Reviewer: {reviewer_name}, Rating: {star_rating}, Title: {review_title}")
                        
                        # Scrape main category
                        main_category_element = driver.find_element(By.XPATH, "//*[@id='wayfinding-breadcrumbs_feature_div']/ul//li/span/a")
                        main_category = main_category_element.text.strip()

                        # Find subcategory using XPath
                        subcategory_element = driver.find_element(By.XPATH, "//ul[contains(@class, 'a-unordered-list')]/li[9]/span/a")
                        subcategory = subcategory_element.text.strip()

                        # Update the subcategory value in the data dictionary
                        data[-1]['subcategory'] = subcategory

                        # Print subcategory
                        print(f"Subcategory: {subcategory}")

                        # Find gender using XPath
                        gender_element = driver.find_element(By.XPATH, "//*[@id='wayfinding-breadcrumbs_feature_div']//ul/li[3]/span")
                        gender = gender_element.text.strip()

                        # Update the gender value in the data dictionary
                        data[-1]['gender'] = gender

                        # Print gender
                        print(f"Gender: {gender}")
                        
                        # Random delay between 3 to 6 seconds
                        time.sleep(random.uniform(3, 6))

                except Exception as e:
                    print(f"Error scraping review details: {e}")

                # Navigate back to the clothing list page to continue the loop
                driver.back()  # Go back to the previous page
                time.sleep(random.uniform(3, 6))

        # Find and click the next page button
        next_button = driver.find_elements(By.CLASS_NAME, 's-pagination-next')
        if next_button:
            driver.execute_script("arguments[0].click();", next_button[0])
            time.sleep(5)  # Adjust based on page load time
        else:
            print("No more pages found")
            break

    # Output data to JSON
    with open('Category1.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

finally:
    driver.quit()
    print(f"Scraped {len(data)} products with reviews, product details, brand site descriptions, image URLs, and prices with pagination support.")


Category = Shoes

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import random

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)

try:
    # Navigate to the main page
    driver.get("https://www.amazon.com")

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "nav-hamburger-menu"))).click()   # Main Dropbar
    time.sleep(2) 

    see_all_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//li/a/div[contains(text(), 'See all')]"))   # See all button
    )
    see_all_button.click()
    time.sleep(2) 

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a/div[contains(text(), \"Men's Fashion\")]"))).click() # Mens fashion button
    time.sleep(2)  

    # Click on "Clothing" using the new provided path
    clothing_link = "/s?i=specialty-aps&bbn=16225019011&rh=n%3A7141123011%2Cn%3A16225019011%2Cn%3A679255011&ref=nav_em__nav_desktop_sa_intl_shoes_0_2_13_3"
    driver.get(f"https://www.amazon.com{clothing_link}")
    time.sleep(5)  # Adjust based on page load time

    data = []
    product_urls = []  # New list to store product URLs
    gender = ""  # Initialize gender before the loop
    main_category = ""
    subcategory = ""  # Initialize subcategory before the loop

    while len(data) < 500:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        products = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for product in products:
            if len(data) >= 500:
                break

            title = product.find('span', class_='a-size-base-plus')
            
            if title:
                product_link = product.find('a', class_='a-link-normal')['href']
                full_link = f"https://www.amazon.com{product_link}"
                product_urls.append(full_link)  # Add product URL to the list
                
                data.append({
                    'product_url': full_link,  # Add the product URL
                    'title': title.text.strip(),
                    'main_category': main_category,
                    'subcategory': subcategory,
                    'gender': gender,
                })
                
                # Click on the product link
                driver.get(full_link)
                time.sleep(3)  # Adjust based on page load time
                
                # Scrape product details using provided XPath
                try:
                    product_details_element = driver.find_element(By.XPATH, "//*[contains(@id, 'detailBullets')]")
                    product_details = product_details_element.text.strip()
                    data[-1]['product_details'] = product_details
                    print(f"Scraped Product Details - Product: {title.text.strip()}, Details: {product_details}")

                    # Scrape product description using provided XPath
                    product_description_element = driver.find_element(By.XPATH, "(//*[contains(@id, 'bylineInfo')])[1]")
                    product_description = product_description_element.text.strip()
                    data[-1]['product_description_brand_site'] = product_description
                    print(f"Scraped Product Description (Brand Site) - Product: {title.text.strip()}, Description: {product_description}")

                    # Scrape product image URLs using provided XPath
                    product_image_elements = driver.find_elements(By.XPATH, "//*[contains(@id, 'a-autoid')]/div/div[1]/img")
                    product_image_urls = [element.get_attribute('src') for element in product_image_elements]
                    data[-1]['product_image_urls'] = product_image_urls
                    print(f"Scraped Product Image URLs - Product: {title.text.strip()}, Image URLs: {product_image_urls}")

                    # Scrape price using provided XPath
                    price_element = driver.find_element(By.XPATH, "//*[@id='corePrice_feature_div']//span")
                    price = price_element.text.strip()
                    data[-1]['price'] = price
                    print(f"Scraped Price - Product: {title.text.strip()}, Price: {price}")

                except Exception as e:
                    print(f"Error scraping product details, brand site description, image URLs, or price: {e}")

                # Scrape review details
                try:
                    reviews = driver.find_elements(By.XPATH, "//div[contains(@data-hook, 'review')]")
                    for review in reviews[:1]:  # Get only the first review
                        reviewer_name_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[1]/a/div[2]/span")
                        reviewer_name = reviewer_name_element.text

                        review_title_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[2]/a/span[2]")
                        review_title = review_title_element.text

                        star_ratings_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[2]/a/i")

                        star_rating_text = star_ratings_element.get_attribute("class")
                        star_rating = star_rating_text if star_rating_text else None

                        data[-1].setdefault('reviews', []).append({
                            'reviewer_name': reviewer_name,
                            'star_rating': star_rating,
                            'review_title': review_title
                        })

                        # Print review details
                        print(f"Scraped Review - Product: {title.text.strip()}, Reviewer: {reviewer_name}, Rating: {star_rating}, Title: {review_title}")
                        
                        # Scrape main category
                        main_category_element = driver.find_element(By.XPATH, "//*[@id='wayfinding-breadcrumbs_feature_div']/ul//li/span/a")
                        main_category = main_category_element.text.strip()

                        # Find subcategory using XPath
                        subcategory_element = driver.find_element(By.XPATH, "//ul[contains(@class, 'a-unordered-list')]/li[9]/span/a")
                        subcategory = subcategory_element.text.strip()

                        # Update the subcategory value in the data dictionary
                        data[-1]['subcategory'] = subcategory

                        # Print subcategory
                        print(f"Subcategory: {subcategory}")

                        # Find gender using XPath
                        gender_element = driver.find_element(By.XPATH, "//*[@id='wayfinding-breadcrumbs_feature_div']//ul/li[3]/span")
                        gender = gender_element.text.strip()

                        # Update the gender value in the data dictionary
                        data[-1]['gender'] = gender

                        # Print gender
                        print(f"Gender: {gender}")
                        
                        # Random delay between 3 to 6 seconds
                        time.sleep(random.uniform(3, 6))

                except Exception as e:
                    print(f"Error scraping review details: {e}")

                # Navigate back to the clothing list page to continue the loop
                driver.back()  # Go back to the previous page
                time.sleep(random.uniform(3, 6))

        # Find and click the next page button
        next_button = driver.find_elements(By.CLASS_NAME, 's-pagination-next')
        if next_button:
            driver.execute_script("arguments[0].click();", next_button[0])
            time.sleep(5)  # Adjust based on page load time
        else:
            print("No more pages found")
            break

    # Output data to JSON
    with open('Category1.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

finally:
    driver.quit()
    print(f"Scraped {len(data)} products with reviews, product details, brand site descriptions, image URLs, and prices with pagination support.")


Scraped Product Details - Product: Isotoner Men's Microsuede Moccasin Slippers - Men’s Indoor and Outdoor Sole Slippers, Features Memory Foam for a Customer Fit and Arch Support for All-Day Comfort, Machine Washable, Details: Product details
Package Dimensions : 12.95 x 6.85 x 4.92 inches; 6.4 ounces
Item model number : 91979
Department : mens
Date First Available : September 28, 2016
Manufacturer : isotoner
ASIN : B01IP7QY36
Best Sellers Rank: #11,327 in Clothing, Shoes & Jewelry (See Top 100 in Clothing, Shoes & Jewelry)
#53 in Men's Slippers
Customer Reviews: 4.2
10,974 ratings
Scraped Product Description (Brand Site) - Product: Isotoner Men's Microsuede Moccasin Slippers - Men’s Indoor and Outdoor Sole Slippers, Features Memory Foam for a Customer Fit and Arch Support for All-Day Comfort, Machine Washable, Description: Visit the isotoner Store
Scraped Product Image URLs - Product: Isotoner Men's Microsuede Moccasin Slippers - Men’s Indoor and Outdoor Sole Slippers, Features Memory 

Category = Watches

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import random

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)

try:
    # Navigate to the main page
    driver.get("https://www.amazon.com")

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "nav-hamburger-menu"))).click()   # Main Dropbar
    time.sleep(2) 

    see_all_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//li/a/div[contains(text(), 'See all')]"))   # See all button
    )
    see_all_button.click()
    time.sleep(2) 

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a/div[contains(text(), \"Men's Fashion\")]"))).click() # Mens fashion button
    time.sleep(2)  

    # Click on "Clothing" using the new provided path
    clothing_link = "/s?i=specialty-aps&bbn=16225019011&rh=n%3A7141123011%2Cn%3A16225019011%2Cn%3A6358539011&ref=nav_em__nav_desktop_sa_intl_watches_0_2_13_4"
    driver.get(f"https://www.amazon.com{clothing_link}")
    time.sleep(5)  # Adjust based on page load time

    data = []
    product_urls = []  # New list to store product URLs
    gender = ""  # Initialize gender before the loop
    main_category = ""
    subcategory = ""  # Initialize subcategory before the loop

    while len(data) < 60:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        products = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for product in products:
            if len(data) >= 60:
                break

            title = product.find('span', class_='a-size-base-plus')
            
            if title:
                product_link = product.find('a', class_='a-link-normal')['href']
                full_link = f"https://www.amazon.com{product_link}"
                product_urls.append(full_link)  # Add product URL to the list
                
                data.append({
                    'product_url': full_link,  # Add the product URL
                    'title': title.text.strip(),
                    'main_category': main_category,
                    'subcategory': subcategory,
                    'gender': gender,
                })
                
                # Click on the product link
                driver.get(full_link)
                time.sleep(3)  # Adjust based on page load time
                
                # Scrape product details using provided XPath
                try:
                    product_details_element = driver.find_element(By.XPATH, "//*[contains(@id, 'detailBullets')]")
                    product_details = product_details_element.text.strip()
                    data[-1]['product_details'] = product_details
                    print(f"Scraped Product Details - Product: {title.text.strip()}, Details: {product_details}")

                    # Scrape product description using provided XPath
                    product_description_element = driver.find_element(By.XPATH, "(//*[contains(@id, 'bylineInfo')])[1]")
                    product_description = product_description_element.text.strip()
                    data[-1]['product_description_brand_site'] = product_description
                    print(f"Scraped Product Description (Brand Site) - Product: {title.text.strip()}, Description: {product_description}")

                    # Scrape product image URLs using provided XPath
                    product_image_elements = driver.find_elements(By.XPATH, "//*[contains(@id, 'a-autoid')]/div/div[1]/img")
                    product_image_urls = [element.get_attribute('src') for element in product_image_elements]
                    data[-1]['product_image_urls'] = product_image_urls
                    print(f"Scraped Product Image URLs - Product: {title.text.strip()}, Image URLs: {product_image_urls}")

                    # Scrape price using provided XPath
                    price_element = driver.find_element(By.XPATH, "//*[@id='corePrice_feature_div']//span")
                    price = price_element.text.strip()
                    data[-1]['price'] = price
                    print(f"Scraped Price - Product: {title.text.strip()}, Price: {price}")

                except Exception as e:
                    print(f"Error scraping product details, brand site description, image URLs, or price: {e}")

                # Scrape review details
                try:
                    reviews = driver.find_elements(By.XPATH, "//div[contains(@data-hook, 'review')]")
                    for review in reviews[:1]:  # Get only the first review
                        reviewer_name_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[1]/a/div[2]/span")
                        reviewer_name = reviewer_name_element.text

                        review_title_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[2]/a/span[2]")
                        review_title = review_title_element.text

                        star_ratings_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[2]/a/i")

                        star_rating_text = star_ratings_element.get_attribute("class")
                        star_rating = star_rating_text if star_rating_text else None

                        data[-1].setdefault('reviews', []).append({
                            'reviewer_name': reviewer_name,
                            'star_rating': star_rating,
                            'review_title': review_title
                        })

                        # Print review details
                        print(f"Scraped Review - Product: {title.text.strip()}, Reviewer: {reviewer_name}, Rating: {star_rating}, Title: {review_title}")
                        
                        # Scrape main category
                        main_category_element = driver.find_element(By.XPATH, "//*[@id='wayfinding-breadcrumbs_feature_div']/ul//li/span/a")
                        main_category = main_category_element.text.strip()

                        # Find subcategory using XPath
                        subcategory_element = driver.find_element(By.XPATH, "//ul[contains(@class, 'a-unordered-list')]/li[9]/span/a")
                        subcategory = subcategory_element.text.strip()

                        # Update the subcategory value in the data dictionary
                        data[-1]['subcategory'] = subcategory

                        # Print subcategory
                        print(f"Subcategory: {subcategory}")

                        # Find gender using XPath
                        gender_element = driver.find_element(By.XPATH, "//*[@id='wayfinding-breadcrumbs_feature_div']//ul/li[3]/span")
                        gender = gender_element.text.strip()

                        # Update the gender value in the data dictionary
                        data[-1]['gender'] = gender

                        # Print gender
                        print(f"Gender: {gender}")
                        
                        # Random delay between 3 to 6 seconds
                        time.sleep(random.uniform(3, 6))

                except Exception as e:
                    print(f"Error scraping review details: {e}")

                # Navigate back to the clothing list page to continue the loop
                driver.back()  # Go back to the previous page
                time.sleep(random.uniform(3, 6))

        # Find and click the next page button
        next_button = driver.find_elements(By.CLASS_NAME, 's-pagination-next')
        if next_button:
            driver.execute_script("arguments[0].click();", next_button[0])
            time.sleep(5)  # Adjust based on page load time
        else:
            print("No more pages found")
            break

    # Output data to JSON
    with open('CategoryWatches.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

finally:
    driver.quit()
    print(f"Scraped {len(data)} products with reviews, product details, brand site descriptions, image URLs, and prices with pagination support.")


Scraped Product Details - Product: BOSS Men's Quartz Chronograph Watch - Modern - Water Resistant, Details: Product details
Product Dimensions : 3.94 x 3.94 x 3.94 inches; 15.52 ounces
Item model number : 1513755
Department : mens
Date First Available : March 2, 2021
Manufacturer : Hugo Boss
ASIN : B00FIVGV86
Country of Origin : China
Best Sellers Rank: #268,741 in Clothing, Shoes & Jewelry (See Top 100 in Clothing, Shoes & Jewelry)
#1,284 in Men's Wrist Watches
Customer Reviews: 4.3
119 ratings
Scraped Product Description (Brand Site) - Product: BOSS Men's Quartz Chronograph Watch - Modern - Water Resistant, Description: Visit the BOSS Store
Scraped Product Image URLs - Product: BOSS Men's Quartz Chronograph Watch - Modern - Water Resistant, Image URLs: ['https://m.media-amazon.com/images/I/31RxXQGFxxL._SS36_.jpg', 'https://m.media-amazon.com/images/I/41TmGCCZMKL._SS36_.jpg', 'https://m.media-amazon.com/images/I/41YD+zzF-GL._SS36_.jpg', 'https://m.media-amazon.com/images/I/41LhgFMW5zL

Category = Mens Accessories

In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import random

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)

try:
    # Navigate to the main page
    driver.get("https://www.amazon.com")

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "nav-hamburger-menu"))).click()   # Main Dropbar
    time.sleep(2) 

    see_all_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//li/a/div[contains(text(), 'See all')]"))   # See all button
    )
    see_all_button.click()
    time.sleep(2) 

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a/div[contains(text(), \"Men's Fashion\")]"))).click() # Mens fashion button
    time.sleep(2)  

    # Click on "Clothing" using the new provided path
    clothing_link = "/s?i=specialty-aps&bbn=16225019011&rh=n%3A7141123011%2Cn%3A16225019011%2Cn%3A2474937011&ref=nav_em__nav_desktop_sa_intl_accessories_0_2_13_5"
    driver.get(f"https://www.amazon.com{clothing_link}")
    time.sleep(5)  # Adjust based on page load time

    data = []
    product_urls = []  # New list to store product URLs
    gender = ""  # Initialize gender before the loop
    main_category = ""
    subcategory = ""  # Initialize subcategory before the loop

    while len(data) < 60:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        products = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for product in products:
            if len(data) >= 60:
                break

            title = product.find('span', class_='a-size-base-plus')
            
            if title:
                product_link = product.find('a', class_='a-link-normal')['href']
                full_link = f"https://www.amazon.com{product_link}"
                product_urls.append(full_link)  # Add product URL to the list
                
                data.append({
                    'product_url': full_link,  # Add the product URL
                    'title': title.text.strip(),
                    'main_category': main_category,
                    'subcategory': subcategory,
                    'gender': gender,
                })
                
                # Click on the product link
                driver.get(full_link)
                time.sleep(3)  # Adjust based on page load time
                
                # Scrape product details using provided XPath
                try:
                    product_details_element = driver.find_element(By.XPATH, "//*[contains(@id, 'detailBullets')]")
                    product_details = product_details_element.text.strip()
                    data[-1]['product_details'] = product_details
                    print(f"Scraped Product Details - Product: {title.text.strip()}, Details: {product_details}")

                    # Scrape product description using provided XPath
                    product_description_element = driver.find_element(By.XPATH, "(//*[contains(@id, 'bylineInfo')])[1]")
                    product_description = product_description_element.text.strip()
                    data[-1]['product_description_brand_site'] = product_description
                    print(f"Scraped Product Description (Brand Site) - Product: {title.text.strip()}, Description: {product_description}")

                    # Scrape product image URLs using provided XPath
                    product_image_elements = driver.find_elements(By.XPATH, "//*[contains(@id, 'a-autoid')]/div/div[1]/img")
                    product_image_urls = [element.get_attribute('src') for element in product_image_elements]
                    data[-1]['product_image_urls'] = product_image_urls
                    print(f"Scraped Product Image URLs - Product: {title.text.strip()}, Image URLs: {product_image_urls}")

                    # Scrape price using provided XPath
                    price_element = driver.find_element(By.XPATH, "//*[@id='corePrice_feature_div']//span")
                    price = price_element.text.strip()
                    data[-1]['price'] = price
                    print(f"Scraped Price - Product: {title.text.strip()}, Price: {price}")

                except Exception as e:
                    print(f"Error scraping product details, brand site description, image URLs, or price: {e}")

                # Scrape review details
                try:
                    reviews = driver.find_elements(By.XPATH, "//div[contains(@data-hook, 'review')]")
                    for review in reviews[:1]:  # Get only the first review
                        reviewer_name_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[1]/a/div[2]/span")
                        reviewer_name = reviewer_name_element.text

                        review_title_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[2]/a/span[2]")
                        review_title = review_title_element.text

                        star_ratings_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[2]/a/i")

                        star_rating_text = star_ratings_element.get_attribute("class")
                        star_rating = star_rating_text if star_rating_text else None

                        data[-1].setdefault('reviews', []).append({
                            'reviewer_name': reviewer_name,
                            'star_rating': star_rating,
                            'review_title': review_title
                        })

                        # Print review details
                        print(f"Scraped Review - Product: {title.text.strip()}, Reviewer: {reviewer_name}, Rating: {star_rating}, Title: {review_title}")
                        
                        # Scrape main category
                        main_category_element = driver.find_element(By.XPATH, "//*[@id='wayfinding-breadcrumbs_feature_div']/ul//li/span/a")
                        main_category = main_category_element.text.strip()

                        # Find subcategory using XPath
                        subcategory_element = driver.find_element(By.XPATH, "//ul[contains(@class, 'a-unordered-list')]/li[9]/span/a")
                        subcategory = subcategory_element.text.strip()

                        # Update the subcategory value in the data dictionary
                        data[-1]['subcategory'] = subcategory

                        # Print subcategory
                        print(f"Subcategory: {subcategory}")

                        # Find gender using XPath
                        gender_element = driver.find_element(By.XPATH, "//*[@id='wayfinding-breadcrumbs_feature_div']//ul/li[3]/span")
                        gender = gender_element.text.strip()

                        # Update the gender value in the data dictionary
                        data[-1]['gender'] = gender

                        # Print gender
                        print(f"Gender: {gender}")
                        
                        # Random delay between 3 to 6 seconds
                        time.sleep(random.uniform(3, 6))

                except Exception as e:
                    print(f"Error scraping review details: {e}")

                # Navigate back to the clothing list page to continue the loop
                driver.back()  # Go back to the previous page
                time.sleep(random.uniform(3, 6))

        # Find and click the next page button
        next_button = driver.find_elements(By.CLASS_NAME, 's-pagination-next')
        if next_button:
            driver.execute_script("arguments[0].click();", next_button[0])
            time.sleep(5)  # Adjust based on page load time
        else:
            print("No more pages found")
            break

    # Output data to JSON
    with open('CategoryAccessories.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

finally:
    driver.quit()
    print(f"Scraped {len(data)} products with reviews, product details, brand site descriptions, image URLs, and prices with pagination support.")


Scraped Product Details - Product: Spyder unisex-adult Cruise TubeTube, Details: Product details
Item model number : 198006
Department : unisex-adult
Date First Available : November 25, 2023
Manufacturer : Spyder
ASIN : B0CNZKNRNJ
Best Sellers Rank: #1,646,869 in Clothing, Shoes & Jewelry (See Top 100 in Clothing, Shoes & Jewelry)
#34,310 in Women's Hats & Caps
#35,972 in Men's Hats & Caps
Customer Reviews: 5.0
1 rating
Scraped Product Description (Brand Site) - Product: Spyder unisex-adult Cruise TubeTube, Description: Visit the Spyder Store
Scraped Product Image URLs - Product: Spyder unisex-adult Cruise TubeTube, Image URLs: ['https://m.media-amazon.com/images/I/31MKjFCs9EL._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/512PFrhHRQL._SX38_SY50_CR,0,0,38,50_.jpg', 'https://m.media-amazon.com/images/I/31HfAeP7qNL._SX38_SY50_CR,0,0,38,50_.jpg']
Scraped Price - Product: Spyder unisex-adult Cruise TubeTube, Price: $29
00
Error scraping review details: Message: no such

Category = Mens Oxfords

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import json
import random

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=chrome_options)

try:
    # Navigate to the main page
    driver.get("https://www.amazon.com")

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID, "nav-hamburger-menu"))).click()   # Main Dropbar
    time.sleep(2) 

    see_all_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.XPATH, "//li/a/div[contains(text(), 'See all')]"))   # See all button
    )
    see_all_button.click()
    time.sleep(2) 

    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a/div[contains(text(), \"Men's Fashion\")]"))).click() # Mens fashion button
    time.sleep(2)  

    # Click on "Clothing" using the new provided path
    clothing_link = "/s?i=fashion-mens-intl-ship&bbn=16225019011&rh=n%3A7141123011%2Cn%3A16225019011%2Cn%3A7147441011%2Cn%3A679255011%2Cn%3A679319011&dc&ds=v1%3AgUO6nZp39zhsgL77SXIbOtciW5DSrKQY%2B7fhayofyh8&qid=1708534068&rnid=7141123011&ref=sr_nr_n_8"
    driver.get(f"https://www.amazon.com{clothing_link}")
    time.sleep(5)  # Adjust based on page load time

    data = []
    product_urls = []  # New list to store product URLs
    gender = ""  # Initialize gender before the loop
    main_category = ""
    subcategory = ""  # Initialize subcategory before the loop

    while len(data) < 60:
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        products = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        for product in products:
            if len(data) >= 60:
                break

            title = product.find('span', class_='a-size-base-plus')
            
            if title:
                product_link = product.find('a', class_='a-link-normal')['href']
                full_link = f"https://www.amazon.com{product_link}"
                product_urls.append(full_link)  # Add product URL to the list
                
                data.append({
                    'product_url': full_link,  # Add the product URL
                    'title': title.text.strip(),
                    'main_category': main_category,
                    'subcategory': subcategory,
                    'gender': gender,
                })
                
                # Click on the product link
                driver.get(full_link)
                time.sleep(3)  # Adjust based on page load time
                
                # Scrape product details using provided XPath
                try:
                    product_details_element = driver.find_element(By.XPATH, "//*[contains(@id, 'detailBullets')]")
                    product_details = product_details_element.text.strip()
                    data[-1]['product_details'] = product_details
                    print(f"Scraped Product Details - Product: {title.text.strip()}, Details: {product_details}")

                    # Scrape product description using provided XPath
                    product_description_element = driver.find_element(By.XPATH, "(//*[contains(@id, 'bylineInfo')])[1]")
                    product_description = product_description_element.text.strip()
                    data[-1]['product_description_brand_site'] = product_description
                    print(f"Scraped Product Description (Brand Site) - Product: {title.text.strip()}, Description: {product_description}")

                    # Scrape product image URLs using provided XPath
                    product_image_elements = driver.find_elements(By.XPATH, "//*[contains(@id, 'a-autoid')]/div/div[1]/img")
                    product_image_urls = [element.get_attribute('src') for element in product_image_elements]
                    data[-1]['product_image_urls'] = product_image_urls
                    print(f"Scraped Product Image URLs - Product: {title.text.strip()}, Image URLs: {product_image_urls}")

                    # Scrape price using provided XPath
                    price_element = driver.find_element(By.XPATH, "//*[@id='corePrice_feature_div']//span")
                    price = price_element.text.strip()
                    data[-1]['price'] = price
                    print(f"Scraped Price - Product: {title.text.strip()}, Price: {price}")

                except Exception as e:
                    print(f"Error scraping product details, brand site description, image URLs, or price: {e}")

                # Scrape review details
                try:
                    reviews = driver.find_elements(By.XPATH, "//div[contains(@data-hook, 'review')]")
                    for review in reviews[:1]:  # Get only the first review
                        reviewer_name_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[1]/a/div[2]/span")
                        reviewer_name = reviewer_name_element.text

                        review_title_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[2]/a/span[2]")
                        review_title = review_title_element.text

                        star_ratings_element = review.find_element(By.XPATH, "//*[contains(@id, 'customer_review')]/div[2]/a/i")

                        star_rating_text = star_ratings_element.get_attribute("class")
                        star_rating = star_rating_text if star_rating_text else None

                        data[-1].setdefault('reviews', []).append({
                            'reviewer_name': reviewer_name,
                            'star_rating': star_rating,
                            'review_title': review_title
                        })

                        # Print review details
                        print(f"Scraped Review - Product: {title.text.strip()}, Reviewer: {reviewer_name}, Rating: {star_rating}, Title: {review_title}")
                        
                        # Scrape main category
                        main_category_element = driver.find_element(By.XPATH, "//*[@id='wayfinding-breadcrumbs_feature_div']/ul//li/span/a")
                        main_category = main_category_element.text.strip()

                        # Find subcategory using XPath
                        subcategory_element = driver.find_element(By.XPATH, "//ul[contains(@class, 'a-unordered-list')]/li[9]/span/a")
                        subcategory = subcategory_element.text.strip()

                        # Update the subcategory value in the data dictionary
                        data[-1]['subcategory'] = subcategory

                        # Print subcategory
                        print(f"Subcategory: {subcategory}")

                        # Find gender using XPath
                        gender_element = driver.find_element(By.XPATH, "//*[@id='wayfinding-breadcrumbs_feature_div']//ul/li[3]/span")
                        gender = gender_element.text.strip()

                        # Update the gender value in the data dictionary
                        data[-1]['gender'] = gender

                        # Print gender
                        print(f"Gender: {gender}")
                        
                        # Random delay between 3 to 6 seconds
                        time.sleep(random.uniform(3, 6))

                except Exception as e:
                    print(f"Error scraping review details: {e}")

                # Navigate back to the clothing list page to continue the loop
                driver.back()  # Go back to the previous page
                time.sleep(random.uniform(3, 6))

        # Find and click the next page button
        next_button = driver.find_elements(By.CLASS_NAME, 's-pagination-next')
        if next_button:
            driver.execute_script("arguments[0].click();", next_button[0])
            time.sleep(5)  # Adjust based on page load time
        else:
            print("No more pages found")
            break

    # Output data to JSON
    with open('CategoryOxfords.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, indent=4)

finally:
    driver.quit()
    print(f"Scraped {len(data)} products with reviews, product details, brand site descriptions, image URLs, and prices with pagination support.")


Scraped Product Details - Product: Cole Haan Men's Lenox Hill Split-Toe Oxford, Details: Product details
Package Dimensions : 12 x 8 x 4 inches; 1.1 Pounds
Item model number : Lenox Hill Split Ox
Department : mens
Date First Available : August 8, 2013
Manufacturer : Cole Haan
ASIN : B00HRO8I5G
Best Sellers Rank: #163,624 in Clothing, Shoes & Jewelry (See Top 100 in Clothing, Shoes & Jewelry)
#340 in Men's Oxfords
Customer Reviews: 4.4
1,390 ratings
Scraped Product Description (Brand Site) - Product: Cole Haan Men's Lenox Hill Split-Toe Oxford, Description: Visit the Cole Haan Store
Scraped Product Image URLs - Product: Cole Haan Men's Lenox Hill Split-Toe Oxford, Image URLs: ['https://m.media-amazon.com/images/I/31Dlfn5wGUL._SS47_.jpg', 'https://m.media-amazon.com/images/I/31t1+uhJMxL._SS47_.jpg']
Scraped Price - Product: Cole Haan Men's Lenox Hill Split-Toe Oxford, Price: $122
02
Scraped Review - Product: Cole Haan Men's Lenox Hill Split-Toe Oxford, Reviewer: user, Rating: a-icon a-ic